In [21]:
import json
import string

In [22]:
left_padding = 6
right_padding = 2

punctuation_error = [
        "%"
    ]

In [23]:
def read_json(path):
    with open(path, 'r') as json_file:
        data = json.load(json_file)
    return data
def write_json(path, data):
    with open(path, 'w') as file:
        # Write the Python data structure as JSON to the file
        json.dump(data, file, indent=2)

def remove_punctuation(text):
    # Create a string of all punctuation characters
    punctuation_chars = string.punctuation

    # Remove punctuation at the beginning and end of the string
    cleaned_text = text.strip(punctuation_chars).strip().strip(punctuation_chars).strip().strip(punctuation_chars).strip()

    return cleaned_text
def span(text, sub_text, s_ind=0):
    sub_text = remove_punctuation(sub_text)
    new_text = text.split()
    if text == sub_text:
        ss = s_ind
        se = s_ind+len(new_text)
        return ss, se, text
    new_sub_text = sub_text
    new_sub_text = new_sub_text.split()
    check = True
    for i in range(len(new_text)):
        temp = i
        for j in range(len(new_sub_text)):
            if new_sub_text[j] != new_text[i + j]:
                if j == len(new_sub_text)-1:
                    for punctuation in punctuation_error:

                        if new_sub_text[j]+punctuation == new_text[i + j]:
                            # print(new_sub_text)
                            new_sub_text[j] = new_sub_text[j]+punctuation 
                            ss = i
                            se = i+j
                            print(new_sub_text)
                            return s_ind+ss, s_ind+se+1, ' '.join(new_sub_text)
                check = False
                break
            else:
                temp = i + j
                check = True
        if check == True:
            ss = i
            se = temp
            return s_ind+ss, s_ind+se+1, sub_text

def convertraw_to_train(data):
    new_data = []
    max_len = 0
    for i in range(len(data)):
        conversation_ID = data[i]['conversation_ID']
        conversation = data[i]['conversation']

        utterance_ID_pool = []
        text_pool = []
        speaker_pool = []
        emotion_pool = {}
        for j in range(len(conversation)):
            utterance_ID_pool.append(conversation[j]['utterance_ID'])
            text_pool.append(conversation[j]['text'])
            if max_len < len(conversation[j]['text'].split()):
                max_len = len(conversation[j]['text'].split())
            speaker_pool.append(conversation[j]['speaker'])
            emotion_pool[j+1] = {}
            emotion_pool[j+1]["property"] = {}
            emotion_pool[j+1]["casual"] = {}
            if "emotion" in conversation[j].keys():
                emotion_pool[j+1]["property"] = conversation[j]['emotion']

        
        if "emotion-cause_pairs" in data[i].keys():
            cause_pairs_pool = []
            cause_pairs = data[i]['emotion-cause_pairs']
            for j in range(len(cause_pairs)):
                emotion_utter = cause_pairs[j][0]
                casual_utter = cause_pairs[j][1]
                utter, emotion = emotion_utter.split("_")
                casual_utter, casual_text = casual_utter.split("_")
                start, end, _ = span(text_pool[int(casual_utter)-1], casual_text)
                emotion_pool[int(utter)]["casual"][int(casual_utter)] = {
                    "casual_text": casual_text,
                    "start": start,
                    "end": end
                }


        for j in range(len(conversation)):
            start_ids = max(0, j-left_padding)
            end_ids = min(j+right_padding+1, len(conversation))
            for k in range(start_ids, end_ids):
                if k not in emotion_pool[j+1]["casual"].keys():
                    emotion_pool[j+1]["casual"][k] = {
                        "casual_text": "",
                        "start": 0,
                        "end": 0
                    }

            emotion_pool[j+1]["casual"] = dict(sorted(emotion_pool[j+1]["casual"].items()))
            start_ids = max(0, j-left_padding)
            end_ids = min(j+right_padding+1, len(conversation))
            paragraph = " ".join(text_pool[start_ids: end_ids])
            
            new_data.append({
                "conversation_ID" : conversation_ID,
                "utterance_ID" : utterance_ID_pool[j],
                "paragraph": paragraph,
                "emotion": emotion_pool[j+1],
                "text_pool": text_pool[start_ids: end_ids],
                "speaker_pool": speaker_pool[start_ids: end_ids],
            })
    print(max_len)
    return new_data

In [24]:
data = read_json("..\data\ECF 2.0\\train\Subtask_1_train.json")
new_data= convertraw_to_train(data)
write_json("..\data\ECF 2.0\\train\\train.json", new_data)

78


In [25]:
data = read_json("..\data\ECF 2.0\\trial\Subtask_1_trial.json")
new_data= convertraw_to_train(data)
write_json("..\data\ECF 2.0\\trial\\trial.json", new_data)

49
